---

<img src="./images/anchormen-logo.png" width="500">

---


# Geopandas Examples & Useful Libraries

In this extra notebook we look at:

- Calculating **distances** between objects
- A spatial data manipulation & visualization **example** using a public shapefile of waterways in Then Netherlands
- The osmxtract library as a good **API for OpenStreetMap** data
- **Folium** as an alternative library for spatial visualization
- **Plotly** as alternative library

## Libraries

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, GeoSeries
from shapely.geometry import Point, LineString, Polygon
import matplotlib.pyplot as plt

%matplotlib inline
pd.options.mode.chained_assignment = None

## Calculating Distances

### Example

In [ ]:
point1 = Point(1.92, 0.47)
point2 = Point(1.33, 1.48)
line1 = LineString([(0.3, 0.6), (1.5, 1.6), (0.9, 2.5)])
line2 = LineString([(0.8, 0.5), (0.5, 1.0), (0.8, 2.1), (1.3, 1.1)])
polygon1 = Polygon([[0, 0], [1, 0], [1, 1], [0, 1]])

print("Point to point: {}".format(
    point1.distance(point2)))

print("Line to point: {}".format(
    line1.distance(point1)))

print("Line to line: {}".format(
    line1.distance(line2)))

print("Polygon to point: {}".format(
    polygon1.distance(point2)))

### Application: given car location, find nearest road

In [ ]:
# Functions

"""
Determine nearest road from Point 'location' and GeoDataFrame 'roads'
"""
def get_nearest_road(roads: GeoDataFrame, location: Point) -> str:
    # Calculate distances of roads to location, select road with minimum distance and retrieve its name
    nearestroad = roads.loc[roads.distance(location).idxmin()]['ROUTE']
    
    return(nearestroad)


"""
Get nearest road from GeoSeries of Point 'locations' and GeoDataFrame 'roads'
"""
def get_nearest_roads(roads: GeoDataFrame, locations: GeoSeries) -> list:
    nearestroads = [get_nearest_road(roads, location) for location in locations]
    
    return(nearestroads)


"""
Plot GeoDataFrame with 'roads' plus GeoSeries with Point 'locations'
"""
def plot_roads_cars(roads: GeoDataFrame, locations: GeoSeries):
    fig, ax = plt.subplots(figsize=(10,10), subplot_kw={'aspect':'equal'})
    ax.set_axis_off()
    roads.plot(ax=ax);
    locations.plot(ax=ax, color='red')


"""
Plot GeoDataFrame with 'roads' plus GeoSeries with Point 'locations'
and optionally highlight nearest roads
"""
def plot_roads_cars(roads: GeoDataFrame, locations: GeoSeries, display_nearest: bool=True):
    # Plot settings
    fig, ax = plt.subplots(figsize=(10,10), subplot_kw={'aspect':'equal'})
    ax.set_axis_off()
    
    # Visualize roads and car locations
    roads.plot(ax=ax, linewidth=.6);
    locations.plot(ax=ax, color='red')
    
    # Calculate nearest roads and visualize (if display_nearest True)
    if(display_nearest):
        ax.set_title('Car locations and nearest roads', size = 16)
        nearest_road_names = get_nearest_roads(roads, locations)
        nearest_roads = roads[roads['ROUTE'].isin(nearest_road_names)]
        nearest_roads.plot(ax=ax, color='red')
    else:
        ax.set_title('Car locations', size = 16)

In [ ]:
# Read roads shapefile

# Shapefile manually downloaded from:
# https://www.rijkswaterstaat.nl/apps/geoservices/geodata/dmc/nwb-wegen/geogegevens/shapefile/NWB-light/
roads_shapefile = './geodata/NWB-Light/nwb-light.shp'

# Standard longitude/latitude coordinate reference system
coordinate_system = 'epsg:4326'

# Load roads shapefile and transform coordinate system
roads = gpd.read_file(roads_shapefile) \
               .to_crs({'init': coordinate_system})

In [ ]:
# Define car locations
car_locations = gpd.GeoSeries([Point(5.009344, 52.138613), Point(5.165653, 52.98645), Point(6.338036, 52.43607)])

In [ ]:
# Get nearest roads for multiple points
get_nearest_roads(roads, car_locations)

In [ ]:
# Visualize
plot_roads_cars(roads, car_locations, display_nearest=True)

## Example: Waterways in The Netherlands

Data Source: Rijkswaterstaat (https://www.rijkswaterstaat.nl/apps/geoservices/geodata/dmc/nwb-vaarwegen/geogegevens/shapefile/01-11-2018/Vaarwegvakken/)

In [ ]:
# Libraries
import urllib
import geopandas as gpd
import matplotlib.pyplot as plt
import mplleaflet

# Shapefile location
base_url = 'https://www.rijkswaterstaat.nl/apps/geoservices/geodata/dmc/nwb-vaarwegen/geogegevens/shapefile/01-11-2018/Vaarwegvakken/'
file_list = ['Vaarwegvakken.shp', 'Vaarwegvakken.prj', 'Vaarwegvakken.shx', 'Vaarwegvakken.dbf']
download_dir = './geodata/tmp/'

# Download shapefile(s)
for file in file_list:
    urllib.request.urlretrieve(url = base_url+file, filename = download_dir + file)
    
# Load shape file and convert to basic coordinate system
gdf = gpd.read_file(download_dir + file_list[0])
gdf_converted = gdf.to_crs({'init': 'epsg:4326'})

# Select coordinate subset using bounding box
lon_min, lon_max = 4.7, 6.7
lat_min, lat_max = 51.3, 52.7
gdf_selection = gdf_converted.cx[lon_min : lon_max, lat_min : lat_max]

In [ ]:
# Mplleaflet visualization
fig, ax = plt.subplots(figsize=(10,10), subplot_kw={'aspect':'equal'})
plot = gdf_selection.plot(column='VAKTYPE', linewidth=2.0, ax=ax)
mplleaflet.display(fig=plot.figure)

## OpenStreetMap API

### geopandas_osm (slow)

In [ ]:
# # Install using: pip install git+https://github.com/jwass/geopandas_osm
# import geopandas_osm.osm
# from shapely.geometry import Polygon

# # Manually define boundary
# #boundary = Polygon([[52.0635223,5.1376253], [52.0675223,5.1376253], [52.0675223,5.1456253], [52.0635223,5.1456253]])

# # Select (small) country
# country = 'Trinidad and Tobago'
# #country = 'Puerto Rico'
# gdf_selected = gdf_countries[gdf_countries.name == country]
# boundary = gdf_selected['geometry'].iloc[0]

# # Download OSM data
# gdf_osm = geopandas_osm.osm.query_osm('way', boundary, recurse='down', tags='highway')

# # Selection
# gdf_osm = gdf_osm[gdf_osm.type == 'LineString'][['highway', 'name', 'geometry']]
# gdf_osm = gdf_osm[gdf_osm.highway.isin(['motorway', 'motorway_link', 'primary', 'primary_link', 'secondary', 'secondary_link'])]
# #gdf_osm = gdf_osm[gdf_osm.highway.isin(['motorway', 'primary'])]

# # Plot
# fig, ax = plt.subplots(figsize=(16,16), subplot_kw={'aspect':'equal'})
# gdf_selected.plot(color='green', ax=ax)
# gdf_osm.plot(column='highway', legend=True, linewidth=2., ax=ax);

### osmxtract (recommended)

OSMxtract is a simple Python package that uses the Overpass API to fetch OpenStreetMap features and, optionally, export them in a GeoJSON file. See: https://pypi.org/project/osmxtract/

In [ ]:
from osmxtract import overpass, location
import geopandas as gpd

# Get bounding box coordinates from a 2km buffer around specific location
lat, lon = location.geocode('Anchormen Amsterdam')
bounds = location.from_buffer(lat, lon, buffer_size=2000)

# Build an overpass QL query and get the JSON response
query = overpass.ql_query(bounds, tag='amenity', values=['cafe', 'bar'])
response = overpass.request(query)

# Parse results as GeoJSON
feature_collection = overpass.as_geojson(response, 'point')

# To GeoPandas GeoDataFrame:
geodataframe = gpd.GeoDataFrame.from_features(feature_collection)
geodataframe.head()

## Folium

In [ ]:
# Visualization functions
import folium 
from folium import plugins

def visualize_map(geodf):
    """Visualize lat/lon locations on map using Folium"""
    
    # Extract lat, lon (for Point geometries)
    geodf['lon'] = geodf.geometry.x
    geodf['lat'] = geodf.geometry.y

    # Start lon/lat
    center_latitude = geodf.lat.mean(skipna=True)
    center_longitude = geodf.lon.mean(skipna=True)
    
    # Initialize map
    map = folium.Map(location = [center_latitude, center_longitude], zoom_start = 12)
 
    # Instantiate a mark cluster object for the locations in the dataframe
    locations = plugins.MarkerCluster().add_to(map)
 
    # Loop through the dataframe and add each data point to the mark cluster
    for lat, lon, name, in zip(geodf.lat, geodf.lon, geodf.name):
        
        folium.Marker(
            location=[lat, lon],
            icon=None,
            popup = folium.Popup(name),
        ).add_to(locations)

    # Display map
    return(map)

In [ ]:
visualize_map(geodataframe)

## Plotly Choropleth Map

From https://plot.ly/python/mapbox-county-choropleth

*Note: To plot on Mapbox maps with Plotly you may need a Mapbox account and a public Mapbox Access Token, depending on which features you are using.*

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})

import plotly.graph_objects as go

fig = go.Figure(go.Choroplethmapbox(geojson=counties, locations=df.fips, z=df.unemp,
                                    colorscale="Viridis", zmin=0, zmax=12,
                                    marker_opacity=0.5, marker_line_width=0))
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=3, mapbox_center = {"lat": 37.0902, "lon": -95.7129})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Spatial joins

In [ ]:
# On MacOSX, this will be needed in order to use the spatial join functionality
#!brew install spatialindex
#!pip install rtree

In [ ]:
# Load datasets
countries = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')) \
    .rename({'name': 'country_name'}, axis=1)
cities = gpd.read_file(gpd.datasets.get_path('naturalearth_cities')) \
    .rename({'name': 'city_name'}, axis=1)

In [ ]:
cities.head()

In [ ]:
# Spatial joins: Which points belong to which region?
cities_with_country = gpd.sjoin(cities, countries, how="inner", op='intersects')
cities_with_country.query('country_name == "Italy"')